In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.optim import lr_scheduler
import cv2
import os
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as  plt 
import numpy as np
import copy
import time


In [12]:
# Set random seeds
torch.manual_seed(42)
torch.cuda.manual_seed(42)

In [13]:
# Setup Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
# Hyper Parameters
num_epochs = 4
batch_size = 4
learning_rate = 0.01

In [15]:
# Define Paths
root = 'B:\Projects\GeoGuide\Task2\\b'
data_dir = os.path.join(root, 'data\events')


In [16]:
# Define Transform
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

In [17]:
# Define Dataset and Dataloader
image_datasets = {x: datasets.ImageFolder(root=os.path.join(data_dir, x), transform=data_transforms[x]) for x in data_transforms}

dataloaders = {x: DataLoader(image_datasets[x], batch_size=batch_size, shuffle=
                            True if x == 'train' else False) for x in data_transforms}

dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}



In [18]:
# Define classes
class_names = image_datasets['train'].classes

In [19]:
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        optimizer.zero_grad()
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model


In [21]:
# Define Model
weights = models.EfficientNet_V2_S_Weights
model = models.efficientnet_v2_s(weights=weights)    
num_ftrs = model.input_shape
model.classifier = torch.nn.Sequential(
    nn.Dropout(p=0.2, inplace=True),
    nn.Linear(in_features=num_ftrs, out_features=len(class_names), bias=True),
    # nn.ReLU6(),   
).to(device)

AttributeError: 'EfficientNet' object has no attribute 'in_features'

In [ ]:
# Setup model and more
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Learning Rate Scheduler
step_lr_scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

In [ ]:
# Train model
model = train_model(model, criterion, optimizer, step_lr_scheduler, num_epochs)

Epoch 0/3
----------
train Loss: 1.9716 Acc: 0.1719
val Loss: 95.3123 Acc: 0.1875

Epoch 1/3
----------
train Loss: 1.7199 Acc: 0.2344
val Loss: 8.6704 Acc: 0.1750

Epoch 2/3
----------
train Loss: 1.7286 Acc: 0.2281
val Loss: 1.6113 Acc: 0.2000

Epoch 3/3
----------
train Loss: 1.7877 Acc: 0.2531
val Loss: 2.2043 Acc: 0.2500

Training complete in 1m 27s
Best val Acc: 0.250000
